In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(
        policy=transforms.AutoAugmentPolicy.CIFAR10
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5,0.5,0.5),
        (0.5,0.5,0.5)
    )
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5,0.5,0.5),
        (0.5,0.5,0.5)
    )
])

In [ ]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    transform=transform_train,
    download=True
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    transform=transform_test,
    download=True
)

100%|██████████| 170M/170M [00:03<00:00, 42.9MB/s]


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model = models.densenet121(pretrained=False)

model.classifier = nn.Linear(
    model.classifier.in_features,
    10
)

model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20
)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = outputs.argmax(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    return running_loss / len(loader), acc

In [ ]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()

    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = outputs.argmax(1)

        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return 100 * correct / total

In [ ]:
best_acc = 0.0
num_epochs = 20

for epoch in range(num_epochs):

    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion
    )

    test_acc = evaluate(model, test_loader)

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Acc:  {train_acc:.2f}%")
    print(f"Test Acc:   {test_acc:.2f}%")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_densenet121.pth")
        print("Best DenseNet model saved!")

    scheduler.step()

print("\n Training Complete")
print(f"Best Test Accuracy: {best_acc:.2f}%")


Epoch [1/20]
Train Loss: 0.5852
Train Acc:  79.48%
Test Acc:   84.14%
Best DenseNet model saved!

Epoch [2/20]
Train Loss: 0.5859
Train Acc:  79.56%
Test Acc:   84.41%
Best DenseNet model saved!

Epoch [3/20]
Train Loss: 0.5908
Train Acc:  79.43%
Test Acc:   84.38%

Epoch [4/20]
Train Loss: 0.6138
Train Acc:  78.78%
Test Acc:   83.21%

Epoch [5/20]
Train Loss: 0.6483
Train Acc:  77.16%
Test Acc:   83.37%

Epoch [6/20]
Train Loss: 0.6933
Train Acc:  75.79%
Test Acc:   81.58%

Epoch [7/20]
Train Loss: 0.7435
Train Acc:  74.27%
Test Acc:   80.51%

Epoch [8/20]
Train Loss: 0.7837
Train Acc:  72.76%
Test Acc:   79.25%

Epoch [9/20]
Train Loss: 0.8242
Train Acc:  71.24%
Test Acc:   74.08%

Epoch [10/20]
Train Loss: 0.8568
Train Acc:  70.02%
Test Acc:   75.36%

Epoch [11/20]
Train Loss: 0.8781
Train Acc:  69.47%
Test Acc:   75.22%

Epoch [12/20]
Train Loss: 0.8929
Train Acc:  68.98%
Test Acc:   74.31%

Epoch [13/20]
Train Loss: 0.8924
Train Acc:  68.90%
Test Acc:   73.87%

Epoch [14/20]
Trai

In [ ]:
torch.save(model.state_dict(), "densenet121_cifar10_clean.pth")